<a href="https://colab.research.google.com/github/thfruchart/tnsi-2020/blob/master/Architecture/Processus/Threading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Découverte du threading

Exécuter plusieurs fois le code suivant.
Expliquer comment se comportent les quatre "threads" de ce programme.

Indication : 

    threading.Thread(target = hello, args = [n])

* crée un nouvel objet **thread**
* qui exécute la fonction :  **hello** (`target= hello` définit le nom de la fonction exécutée par le thread)
* avec pour **argument**(s) : n (`args = [n]` définit la liste des arguments)

## Comptage en parallèle

In [ ]:
import threading

def hello(n):
    for i in range(5):
        print('  '*n, "Thread n°", n, " => i =", i)
    print('  '*n, "----- FIN du thread n°", n)

for n in range(4):
    t = threading.Thread(target = hello, args = [n])
    t.start()


#### L'exécution peut produire différents résultats, par exemple : 

    Thread n° 0  => i = 0
    Thread n° 0  => i = 1
    Thread n° 0  => i = 2
    Thread n° 0  => i = 3
    Thread n° 0  => i = 4
    ----- FIN du thread n° 0
    Thread n° 1  => i = 0
    Thread n° 1     Thread n° 2  => i =  => i = 1
    0
        Thread n°     Thread n° 2  => i = 1
        Thread n° 2  => i = 2
        Thread n° 2  => i = 3
        Thread n° 2  => i = 4
        ----- FIN du thread n° 2
    Thread n° 1  => i = 2
    Thread n° 1  => i = 3
    Thread n° 1  => i = 4
    ----- FIN du thread n° 1
    3  => i = 0
        Thread n° 3  => i = 1
        Thread n° 3  => i = 2
        Thread n° 3  => i = 3
        Thread n° 3  => i = 4
        ----- FIN du thread n° 3

# Notion de verrou

On souhaite éviter les affichage du type : 

    Thread n° Thread n° 3  => i =2  => i = 

Autrement dit, on souhaite garantir que l'exécution de chaque thread ne sera pas interrompue **pendant** l'instruction d'affichage.

Il est possible pour cela d'utiliser des verrous.

* un verrou est défini avec `threading.Lock()`
* chaque verrou ne peut être acquis que par un **unique** thread. 
* une fois défini, un verrou possède les deux méthodes : 
   * `acquire()` qui affecte le verrou et bloque les autres tentatives  pour acquérir ce verrou.
   * `release()` qui libère le verrou et autorise donc une autre tentative  pour acquérir ce verrou.

#### Comparer le programme 1

In [ ]:
import threading

verrou = threading.Lock()  #création d'un verrou

print("tentative d'acquisition")
verrou.acquire()
print("verrou acquis")
print("tentative d'acquisition")
verrou.acquire() 
print("verrou acquis")

#### et le programme 2

In [ ]:
import threading

verrou = threading.Lock()  #création d'un verrou

print("tentative d'acquisition")
verrou.acquire()
print("verrou acquis")
verrou.release()
print("verrou libéré !")
print("tentative d'acquisition")
verrou.acquire()
print("verrou acquis")

## Comptage en parallèle avec verrou

In [ ]:
import threading

verrou = threading.Lock()  #création d'un verrou

def hello(n):
    for i in range(5):
        verrou.acquire()  # acquisition du verrou
        print('  '*n, "Thread n°", n, " => i =", i)
        verrou.release()  # libération du verrou
    # fin de la boucle for

    verrou.acquire()  # acquisition du verrou
    print("----- FIN du thread n°", n)
    verrou.release()  # libération du verrou

for n in range(4):
    t = threading.Thread(target = hello, args = [n])
    t.start()


#### L'exécution peut produire différents résultats. On peut obtenir par exemple :




    Thread n° 0  => i = 0
    Thread n° 0  => i = 1
    Thread n° 0  => i = 2
    Thread n° 0  => i = 3
    Thread n° 0  => i = 4
    ----- FIN du thread n° 0
      Thread n° 1  => i = 0
      Thread n° 1  => i = 1
      Thread n° 1  => i = 2
      Thread n° 1  => i = 3
          Thread n° 3  => i = 0
          Thread n° 3  => i = 1
          Thread n° 3  => i = 2
          Thread n° 3  => i = 3
          Thread n° 3  => i = 4
          ----- FIN du thread n° 3
      Thread n° 1  => i = 4
      ----- FIN du thread n° 1
        Thread n° 2  => i = 0
        Thread n° 2  => i = 1
        Thread n° 2  => i = 2
        Thread n° 2  => i = 3
        Thread n° 2  => i = 4
        ----- FIN du thread n° 2

# Interblocage

Le code suivant crée deux verrous.

Expliquer pourquoi il peut conduire à une situation d'interblocage. 

In [ ]:
import threading

verrou1 = threading.Lock()
verrou2 = threading.Lock()

def commande1():
    print("1. Tentative d'acquisition du verrou 1 ")
    verrou1.acquire()
    print("1. Verrou 1 acquis")
    print("1. Tentative d'acquisition du verrou 2 ")
    verrou2.acquire()
    print("1. Verrou 2 acquis")
    print('1.  -- OK -- ')
    verrou2.release()
    print("1. Verrou 2 libéré")
    verrou1.release()
    print("1. Verrou 1 libéré")


def commande2():
    print("  2. Tentative d'acquisition du verrou 2 ")
    verrou2.acquire()
    print("  2. Verrou 2 acquis")
    print("  2. Tentative d'acquisition du verrou 1 ")
    verrou1.acquire()
    print("  2. Verrou 1 acquis")
    print('  2.  -- OK -- ')
    verrou1.release()
    print("  2. Verrou 1 libéré")
    verrou2.release()
    print("  2. Verrou 2 libéré")


t1 = threading.Thread(target = commande1, args = [])
t2 = threading.Thread(target = commande2, args = [])
t1.start()
t2.start()






#### Exemple de résultat


```
1. Tentative d'acquisition du verrou 1   
  2. Tentative d'acquisition du verrou 2 
  2. Verrou 2 acquis
1. Verrou 1 acquis
1. Tentative d'acquisition du verrou 2 
  2. Tentative d'acquisition du verrou 1
```

